#Predicting future spending to enhance business performance

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import the transformed data.

In [ ]:
# save the transformed dataframe
df_transformed = pd.read_csv('/content/drive/MyDrive/Dataset/Dataset/Dataset/df_transformed_with_customer_id.csv')

In [ ]:
df_transformed.head(2)
df_transformed.drop(['index'], axis=1)

,cc_num,datetime,acct_num,trans_num,unix_time,category,amt,is_fraud,merchant,merch_lat,...,long,city_pop,job,dob,transactions_past_5min,transactions_past_30min,transactions_past_24H,date_of_birth,age,customer_merchant_distance_km
0,213140433504880,2018-12-31 13:00:19,280173576486,a410f9d17a82d616d248d0e65c58420f,1546261219,gas_transport,209.83,0,peterson llc,39.665260,...,-76.0345,213013,fisheries officer,1989-03-06,1.0,1.0,1.0,1989-03-06,34,95.585795
1,571945341956,2018-12-31 13:00:23,41019526172,ba8e37ae12a9b7fe62ed317523c1b702,1546261223,gas_transport,110.47,0,"perez, young and stanley",29.833943,...,-95.1317,31086,"conservation officer, historic buildings",1956-05-05,1.0,1.0,1.0,1956-05-05,67,62.317054
2,372853533998243,2018-12-31 13:01:37,467846623753,bf3e9ab9554652fcd8e0a50ba8dd1d91,1546261297,gas_transport,9.53,0,trujillo-foley,29.400998,...,-98.6895,1595797,"geologist, wellsite",1985-04-12,1.0,1.0,1.0,1985-04-12,38,2.006383
3,676305937770,2018-12-31 13:02:43,661384884419,f303afd08e32a5046250692754d73c6c,1546261363,gas_transport,199.74,0,hatfield group,34.317132,...,-117.2290,46239,"designer, television/film set",1929-11-07,1.0,1.0,1.0,1929-11-07,94,120.976584
4,3547660728326849,2018-12-31 13:04:07,993909782929,80c81bb9612fedaf4eda55e63da5afeb,1546261447,gas_transport,5.16,0,hall plc,38.912612,...,-121.6611,76660,"engineer, mining",1964-08-12,1.0,1.0,1.0,1964-08-12,59,59.269796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4319827,6570703850360591,2022-12-31 12:59:28,182563338256,70dd383dc66f82e20fbf52d37a5de1f5,1672491568,travel,24.43,0,hall ltd,37.400485,...,-76.3004,242803,medical technical officer,1987-03-23,2.0,2.0,14.0,1987-03-23,36,71.128710
4319828,4946209355610391,2022-12-31 12:59:29,232488012574,d88c51c2dc5db1ccdeb3adf58d570c9f,1672491569,travel,21.51,0,"hoffman, gilbert and burns",42.237535,...,-71.9541,8244,information systems manager,1929-04-18,1.0,1.0,9.0,1929-04-18,94,91.850894
4319829,4414925316498286,2022-12-31 12:59:29,395217487731,ede957e0544e56eead0b9361953e9a4d,1672491569,travel,119.89,0,"rhodes, anderson and pratt",26.514762,...,-80.3450,181438,quarry manager,1993-08-07,1.0,1.0,18.0,1993-08-07,30,97.418117
4319830,340534240891645,2022-12-31 12:59:36,160497086732,27af14fe04c632b2ad87545f461c38f7,1672491576,travel,12.84,0,"baker, hunter and brown",34.410350,...,-118.1480,471952,politician's assistant,1971-07-18,1.0,1.0,18.0,1971-07-18,52,97.249090


# Prepare data

In [ ]:
df_transformed['datetime'] = pd.to_datetime(df_transformed['datetime'])

In [ ]:
# Copy the data into a new DataFrame
df = df_transformed.copy()
df['date'] = pd.to_datetime(df_transformed['datetime'].dt.date)

# Group the data by 'date' and 'category', and calculate the aggregations
grouped_df = df.groupby(['date', 'cc_num']).agg({
    'amt': 'sum',
    'trans_num': 'count',
    'age': 'max',
    'city_pop': 'max'
}).reset_index()

In [ ]:
# Rename the columns
grouped_df.columns = ['Date', 'cc_num', 'Total Spend', 'Transaction Count', 'age', 'city_pop']

In [ ]:
grouped_df.sample()

,Date,cc_num,Total Spend,Transaction Count,age,city_pop
495019,2020-11-14,30066149828241,12.46,2,63,900273


## Feature engineering

In [ ]:
grouped_df['month'] = grouped_df['Date'].dt.to_period('M').dt.to_timestamp()

### Calculate total spend and transaction count of each customer in the past interval.

In [ ]:
intervals = ['30D', '182D', '365D']
for interval in intervals:
    for metric in ['Total Spend', 'Transaction Count']:
        grouped_df = grouped_df.set_index('Date')
        grouped_df = grouped_df.sort_index()
        # group by customer and create a rolling window of the past X minutes
        rolling_window = grouped_df.groupby('cc_num')[metric].rolling(interval)
        sum_spending = rolling_window.sum()
        sum_spending = sum_spending.reset_index()
        sum_spending.rename(columns={metric: f'{metric}_past_{interval}'}, inplace=True)
        grouped_df = grouped_df.merge(sum_spending, on=['cc_num', 'Date'])

In [ ]:
grouped_df.sample()

,cc_num,Date,Total Spend,Transaction Count,age,city_pop,month,Total Spend_past_30D,Transaction Count_past_30D,Total Spend_past_182D,Transaction Count_past_182D,Total Spend_past_365D,Transaction Count_past_365D
105305,2237702851961153,2019-05-31,600.89,5,49,88252,2019-05-01,8563.84,80.0,42243.21,390.0,42243.21,390.0


### Transform past spending behaviors into different features.

In [ ]:
# Group the data by month and year, and calculate the sum of total spend
time_df = grouped_df.groupby(['month', 'cc_num']).agg(
    {'Total Spend': 'sum', 
     'Transaction Count':'sum',
     'Total Spend_past_30D': 'mean',
     'Transaction Count_past_30D': 'mean',
     'Total Spend_past_182D': 'mean',
     'Transaction Count_past_182D': 'mean',
     'Total Spend_past_365D': 'mean',
     'Transaction Count_past_365D': 'mean'
     })
time_df = time_df.sort_values(by=['month'])

# Calculate the last month spend and last two month spend
for x in [1, 2, 3]:
    time_df[f'last_{x}_month_spend'] = time_df.groupby('cc_num')['Total Spend'].shift(x)
    time_df[f'last_{x}_month_transaction_count'] = time_df.groupby('cc_num')['Transaction Count'].shift(x)
    time_df[f'last_{x}_month_spend_per_tran'] = time_df[f'last_{x}_month_spend']/time_df[f'last_{x}_month_transaction_count']

    if x > 1:
        time_df[f'last_{x}_month_spend'].fillna(time_df['Total Spend'].mean(), inplace=True)
        time_df[f'last_{x}_month_transaction_count'].fillna(time_df['Transaction Count'].mean(), inplace=True)
        time_df[f'last_{x}_month_spend_per_tran'].fillna(time_df[f'last_{x}_month_spend_per_tran'].mean(), inplace=True)



### Get the next month spend as the target variable.

In [ ]:
# get total spend of next month - our target variable
time_df['next_month_spend'] = time_df.groupby('cc_num')['Total Spend'].shift(-1)

time_df = time_df.reset_index()
time_df.dropna(inplace=True)
time_df.head(2)

,month,cc_num,Total Spend,Transaction Count,Total Spend_past_30D,Transaction Count_past_30D,Total Spend_past_182D,Transaction Count_past_182D,Total Spend_past_365D,Transaction Count_past_365D,last_1_month_spend,last_1_month_transaction_count,last_1_month_spend_per_tran,last_2_month_spend,last_2_month_transaction_count,last_2_month_spend_per_tran,last_3_month_spend,last_3_month_transaction_count,last_3_month_spend_per_tran,next_month_spend
559,2019-01-01,4203265097753962,9925.70,76,5281.082069,44.034483,5317.333103,44.37931,5317.333103,44.37931,354.35,3.0,118.116667,6796.609809,98.621798,69.381332,6796.609809,98.621798,69.514481,8743.33
560,2019-01-01,4208581745273533,14730.47,120,8203.846333,66.966667,8269.334000,67.40000,8269.334000,67.40000,780.18,5.0,156.036000,6796.609809,98.621798,69.381332,6796.609809,98.621798,69.514481,16529.49


## Split data

We want to use past spending behaviors to predict future's spending. 
Therefore, we split the data using transaction month.
- 80% of historical data is used for training.
- The remaining 20% data is used for validation and testing, which are split randomly with 50-50 ratio.

In [ ]:
target = 'next_month_spend'
data_to_fit = time_df.copy().sort_values(by='month')

train_ratio = 0.8
val_test_ratio = 0.2

num_rows = data_to_fit.shape[0]
train_split = int(train_ratio * num_rows)
train_df = data_to_fit.iloc[:train_split].reset_index(drop=True)
val_test_df = data_to_fit.iloc[train_split:].reset_index(drop=True)
print(f"Train data transaction date range: {train_df['month'].min()} - {train_df['month'].max()}")
print(f"Validation-test data transaction date range: {val_test_df['month'].min()} - {val_test_df['month'].max()}")

# drop uncessary columns
train_df.drop(['month', 'cc_num'], axis=1, inplace=True)
features = [f for f in train_df.columns if f!= target]

# randomly split val and test dataset
x_val, x_test, y_val, y_test = train_test_split(val_test_df[features], val_test_df[target], test_size=0.5, random_state=3)

Train data transaction date range: 2019-01-01 00:00:00 - 2022-03-01 00:00:00
Validation-test data transaction date range: 2022-03-01 00:00:00 - 2022-11-01 00:00:00


In [ ]:
y_train = train_df[target]
x_train = train_df[features].copy()

# combine x_train and x_val into predefined train/ val set for Automatic hyperparameter tuning
x_train_copy = x_train.copy()
x_val_copy = x_val.copy()
x_train_copy['test_fold'] = -1
x_val_copy['test_fold'] = 0
x_train_val = pd.concat([x_train_copy, x_val_copy], axis=0)
y_train_val = pd.concat([y_train, y_val], axis=0)

test_fold = x_train_val['test_fold'].values
ps = PredefinedSplit(test_fold=test_fold)

print(f'Train data: {x_train.shape}')
print(f'Val data: {x_val.shape}')
print(f'Test data: {x_test.shape}')

Train data: (33475, 17)
Val data: (4184, 17)
Test data: (4185, 17)


In [ ]:
features

['Total Spend',
 'Transaction Count',
 'Total Spend_past_30D',
 'Transaction Count_past_30D',
 'Total Spend_past_182D',
 'Transaction Count_past_182D',
 'Total Spend_past_365D',
 'Transaction Count_past_365D',
 'last_1_month_spend',
 'last_1_month_transaction_count',
 'last_1_month_spend_per_tran',
 'last_2_month_spend',
 'last_2_month_transaction_count',
 'last_2_month_spend_per_tran',
 'last_3_month_spend',
 'last_3_month_transaction_count',
 'last_3_month_spend_per_tran']

# Assess the baseline

In [ ]:
y_pred = val_test_df['Total Spend']
y_true = val_test_df[target]
baseline_mae = mean_squared_error(y_true, y_pred, squared=False)
print(baseline_mae)

6171.720840274865


# Modeling - Elastic Net

In [ ]:
scaler = preprocessing.StandardScaler(with_mean=True)

## Tune hyperparmeters manually

In [ ]:
alphas = [0.00099, 0.0099, 0.001, 0.0001, 1, 10, 100]
for alpha in alphas:
    print(f'Experimenting alpha = {alpha}...')
    elastic_reg = ElasticNet(
        alpha=alpha, 
        max_iter=10000,
        random_state=43, 
        selection='random')
    
    reg_pipe = Pipeline(
        steps=[
            ("scaler", scaler),
            ("reg", elastic_reg)
        ]
    )
    reg_pipe.fit(x_train, y_train)
    y_train_pred = reg_pipe.predict(x_train)
    msa_train = mean_squared_error(y_train, y_train_pred, squared=False)
    y_val_pred = reg_pipe.predict(x_val)
    msa_val = mean_squared_error(y_val, y_val_pred, squared=False)
    print(f'RMSE train: {msa_train}')
    print(f'RMSE val: {msa_val}')
    print('---')

Experimenting alpha = 0.00099...
RMSE train: 3328.3334569034646
RMSE val: 5698.197012056018
---
Experimenting alpha = 0.0099...
RMSE train: 3328.7704898758006
RMSE val: 5689.254055997893
---
Experimenting alpha = 0.001...
RMSE train: 3328.333737595724
RMSE val: 5698.176491620379
---
Experimenting alpha = 0.0001...
RMSE train: 3328.318075809514
RMSE val: 5700.2508803700775
---
Experimenting alpha = 1...
RMSE train: 3364.582606007403
RMSE val: 5675.026303372029
---
Experimenting alpha = 10...
RMSE train: 3618.6707756694564
RMSE val: 6258.0889520093015
---
Experimenting alpha = 100...
RMSE train: 4507.999648436872
RMSE val: 8325.944991921007
---


In [ ]:
l1_ratios = [0.1, 0.3, 0.5, 0.7, 0.9]
for l1_ratio in l1_ratios:
    print(f'Experimenting l1_ratio = {l1_ratio}...')
    elastic_reg = ElasticNet(
        alpha=0.0099, 
        l1_ratio=l1_ratio,
        max_iter=10000,
        random_state=43, 
        selection='random')
    
    reg_pipe = Pipeline(
        steps=[
            ("scaler", scaler),
            ("reg", elastic_reg)
        ]
    )
    reg_pipe.fit(x_train, y_train)
    y_train_pred = reg_pipe.predict(x_train)
    msa_train = mean_squared_error(y_train, y_train_pred, squared=False)
    y_val_pred = reg_pipe.predict(x_val)
    msa_val = mean_squared_error(y_val, y_val_pred, squared=False)
    print(f'RMSE train: {msa_train}')
    print(f'RMSE val: {msa_val}')
    print('---')

Experimenting l1_ratio = 0.1...
RMSE train: 3329.178744637148
RMSE val: 5686.323997666384
---
Experimenting l1_ratio = 0.3...
RMSE train: 3328.9752889971387
RMSE val: 5687.555544448854
---
Experimenting l1_ratio = 0.5...
RMSE train: 3328.7704898758006
RMSE val: 5689.254055997893
---
Experimenting l1_ratio = 0.7...
RMSE train: 3328.5641427089176
RMSE val: 5691.801550553107
---
Experimenting l1_ratio = 0.9...
RMSE train: 3328.369337879867
RMSE val: 5696.339395961857
---


## Automatic hyperparameter tuning

In [ ]:
scoring = 'neg_root_mean_squared_error'

reg_pipe = Pipeline(
    steps=[
        ("scaler", scaler),
        ("reg", ElasticNet(random_state=43, max_iter=10000))
    ]
)

# define search
parameters = {
    'reg__alpha': [0.0099, 0.0001, 1],
    'reg__l1_ratio': [0.1, 0.3, 0.5],
    'reg__selection': ['cyclic', 'random']
}

search = GridSearchCV(
    estimator=reg_pipe,
    param_grid=parameters,
    cv=ps, # predefined split
    scoring=scoring,
    return_train_score=True
)
# execute search
result = search.fit(x_train_val, y_train_val)

print('est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))

best_est = result.best_estimator_

est=-5675.026, cfg={'reg__alpha': 1, 'reg__l1_ratio': 0.5, 'reg__selection': 'cyclic'}


In [ ]:
best_est.fit(x_train, y_train)
y_train_pred = best_est.predict(x_train)
rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
y_val_pred = best_est.predict(x_val)
rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)
print(f'RMSE train: {rmse_train}')
print(f'RMSE val: {rmse_val}')

RMSE train: 3364.58256044963
RMSE val: 5675.026253073252


In [ ]:
perfect = alt.Chart(
    pd.DataFrame({'target': y_val, 'preds': y_val}),
    ).mark_line(
        color='green'
    ).encode(
        x='target',
        y='preds'
    )

pred_chart = alt.Chart(pd.DataFrame({'target': y_val, 'preds': y_val_pred})).mark_line().encode(
    x='target',
    y='preds'
  )
chart = pred_chart + perfect
chart

alt.LayerChart(...)

# Model evaluation

In [ ]:
best_est.fit(x_train_val[features], y_train_val)
y_train_val_pred = best_est.predict(x_train_val[features])
rmse_train = mean_squared_error(y_train_val, y_train_val_pred, squared=False)
y_test_pred = best_est.predict(x_test[features])
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
print(f'RMSE train: {rmse_train}')
print(f'RMSE test: {rmse_test}')

RMSE train: 3686.712795740782
RMSE test: 4979.393312497075


In [ ]:
perfect = alt.Chart(
    pd.DataFrame({'target': y_test, 'preds': y_test}),
    ).mark_line(
        color='green'
    ).encode(
        x='target',
        y='preds'
    )

pred_chart = alt.Chart(pd.DataFrame({'target': y_test, 'preds': y_test_pred})).mark_line().encode(
    x='target',
    y='preds'
  )
chart = pred_chart + perfect
chart

alt.LayerChart(...)

#Train Ridge Model

In [ ]:
# Solution
from sklearn.linear_model import Ridge

In [ ]:
#Instantiate the Ridge model into a variable called ridge_reg and fit it on the training set
# Solution
ridge_reg = Ridge()
ridge_reg.fit(x_train, y_train)

Ridge()

Print the RMSE and MAE scores for the training set

In [ ]:
# Solution
y_train_preds = ridge_reg.predict(x_train)


In [ ]:
print(mean_squared_error(y_train, y_train_preds, squared=False))


3328.3178783023604


Print the RMSE and MAE scores for the testing set

In [ ]:
# Solution
y_test_preds = ridge_reg.predict(x_test)
print(mean_squared_error(y_test, y_test_preds, squared=False))


5077.750881691956
